<a href="https://colab.research.google.com/github/muhamadrefo/ImageClassificationColab/blob/master/NaiveBayesClassifier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importing Packages

import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from matplotlib.colors import ListedColormap
from pandas.plotting import scatter_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
%cd /content/drive/My Drive/Batik/

/content/drive/My Drive/Batik


#Reading dataset

In [26]:
#col_names = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
#datatrain = pd.read_csv('DataTraining.csv', names=col_names)
#data_train = pd.read_csv('DataTraining.csv') #DataTraining.csv nor DataValidation.csv
data_train = pd.read_csv('DataValidation.csv')
pd.set_option('display.max_columns', 100)
data_train.head(15) #display first 15 data

,Path,Dissimilarity,Contrast,Homogeneity,Energy,Correlation,ASM,Label
0,CNN/DataValid/Tulis/Tulis (34).jpg,0.671184,1.150084,0.711659,0.331302,0.726917,0.109761,Tulis
1,CNN/DataValid/Tulis/Tulis (33).jpg,0.638953,1.014994,0.717791,0.309219,0.816454,0.095616,Tulis
2,CNN/DataValid/Tulis/Tulis (12).jpg,0.322329,0.438000,0.850374,0.617532,0.590479,0.381346,Tulis
3,CNN/DataValid/Tulis/Tulis (4).jpg,0.394859,0.547646,0.817798,0.573175,0.557173,0.328530,Tulis
4,CNN/DataValid/Tulis/Tulis (9).jpg,0.482443,0.787296,0.788360,0.500893,0.621752,0.250894,Tulis
5,CNN/DataValid/Tulis/Tulis (10).jpg,0.368013,0.520199,0.831124,0.571786,0.623596,0.326939,Tulis
6,CNN/DataValid/Tulis/Tulis (8).jpg,0.446268,0.727298,0.804201,0.514562,0.660955,0.264774,Tulis
7,CNN/DataValid/Tulis/Tulis (27).jpg,0.510790,0.705577,0.763781,0.393011,0.666016,0.154458,Tulis
8,CNN/DataValid/Tulis/Tulis (28).jpg,0.668061,1.211943,0.719309,0.352585,0.696101,0.124316,Tulis
9,CNN/DataValid/Tulis/Tulis (5).jpg,0.406710,0.561459,0.812044,0.546859,0.563623,0.299055,Tulis


In [27]:
# show columns
data_train.columns 

Index(['Path', 'Dissimilarity', 'Contrast', 'Homogeneity', 'Energy',
       'Correlation', 'ASM', 'Label'],
      dtype='object')

# Encoding the Categorial Data

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = data_train.apply(le.fit_transform)
df.head(15)

,Path,Dissimilarity,Contrast,Homogeneity,Energy,Correlation,ASM,Label
0,33,94,92,9,18,52,18,0
1,32,92,89,11,12,84,12,0
2,17,10,18,94,104,15,104,0
3,39,22,29,82,103,10,103,0
4,94,56,71,58,87,27,87,0
5,1,20,22,85,102,30,102,0
6,83,38,57,70,93,40,93,0
7,29,66,52,36,40,42,40,0
8,30,93,94,12,24,48,24,0
9,50,24,36,75,100,11,100,0


# Splitting the Data into Training and Testing Data

In [29]:
from sklearn.model_selection import train_test_split as tt
X = df.iloc[:, 1:6].values
Y = df.iloc[:, 7].values
X_train, X_test, Y_train, Y_test = tt(X,Y, test_size=0.2, random_state=4)

In [30]:
X_train.shape

(84, 5)

In [31]:
Y.shape

(105,)

#Naive Bayes Model Start Here

In [32]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,Y_train)
y_pred = nb.predict(X_test)

In [33]:
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
cm = confusion_matrix(Y_test,y_pred)

In [34]:
print("Confusion Matrix: \n",cm)

Confusion Matrix: 
 [[21]]


In [35]:
print("Classification Report: \n", classification_report(Y_test, y_pred))

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        21

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21



In [36]:
print("Accuracy Score: ", accuracy_score(Y_test, y_pred))

Accuracy Score:  1.0
